In [1]:
import tkinter
from tkinter import * 
import tkinter.messagebox

import ttkbootstrap as ttk
from ttkbootstrap.constants import *

from PIL import Image, ImageTk

import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np

import geocoder
from geopy.geocoders import Nominatim

In [2]:
# This block is used for save user's current location as a png.

# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
# from PIL import Image, ImageTk

# # set up options for headless browsing
# chrome_options = Options()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--disable-gpu')

# # start a new Chrome browser
# driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# # load the HTML file
# driver.get('file:/Users/sunnytyw/Desktop/chicago_map.html')

# # take a screenshot of the page
# driver.save_screenshot('screenshot.png')

# # close the browser
# driver.quit()

# # open the screenshot and save it as a PNG
# with Image.open('screenshot.png') as img:
#     img.save('output.png', 'png')



In [3]:
# get user's current longtitude and latitude 
g = geocoder.ip('me')
geo_list = g.latlng #43.0775, -89.3831
geo_list=[41.810,-87.713]

In [4]:
# transfer longtitude and latitude to the corresponding zipcode
def get_zipcode(lat, long):
    geolocator = Nominatim(user_agent="http")
    location = geolocator.reverse(f"{lat}, {long}")
    address = location.raw['address']
    zipcode = address.get('postcode')
    return zipcode

# code = int(get_zipcode(geo_list[0], geo_list[1]))
# code = 60628 #since rn we're not at Chicago, we just assume a randon zipcode that located in Chicago

In [5]:
# How many miles away from the crush area (int).
df = pd.read_csv("merged.csv")

def haversine(lat1, lon1, lat2, lon2):
    # Convert coordinates to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Calculate difference between latitudes and longitudes
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calculate Haversine formula
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of earth in kilometers

    # Return distance in kilometers
    distance = c * r
    return distance



for index, row in df.iterrows():
    distance = haversine(geo_list[0], geo_list[1],row['Lat'],row['Long'])
    if distance<=1.61:
        harre=int(row['zip_code'])
        index+=1
        dangerous_rank = 'You are in #' + str(index) + ' hazard zone.'
        distance= round(distance,3)
        break

In [6]:
#create a message box when user want to know the dangerous rank of his/hers location
def btnClick():
    tkinter.messagebox.showinfo("Ranking", dangerous_rank)

In [7]:
#visualization

#window = Tk()
#window = ttk.Window()
window = Tk()

#style = ttk.Style()
#style = tk.Style()

#style.theme_use("superhero")# set theme for output

window.title("Safer Roads Initiative") #tittle
window.geometry("325x500") #set the size of canvas
window.iconphoto(False, ttk.PhotoImage(file = 'car3.png')) #create a icon for the output

title_label = Label(window, text = 'Road Safety Alert System', font = ('Futura', 25)).pack()#content of text
title_label1 = Label(window, text = 'Chicago', font = ('Futura', 25)).pack()
#speed_label = Label(window, text = 'Do Not Exceed ' + str(speed_lim) + ' mph', font = ('Futura', 23)).pack()

if distance / 1.61 <= 1:
    tkinter.messagebox.showwarning("warning", "You are "+ str(distance) + " mile(s) away from crash area") #create a messagebox
    speed_label = Label(window, text = "You are in dangerous area", font = ('Futura', 24), fg = 'red').pack()
    
    safe_button = Button(window, text = "Am I Safe?", command = btnClick).pack() #create a button
    
    #dangerous_label = Label(window, text = dangerous_rank, font = ('Futura', 15)).pack()
    
map_pic = PhotoImage(file = 'shiyan.png')
img = Image.open('shiyan.png')
photo = ImageTk.PhotoImage(img)
imglabel = Label(window, image = photo).pack(side = BOTTOM, pady = (0,1)) #insert a picture in image

#lable1.grid(column=1, row=1)
window.mainloop()